In [1]:
# 💡 Note: Runs this jupyter notebook from the projects root, mimicking a root run.
import os
os.chdir(os.getcwd().rsplit("/", 1)[0])
print(os.getcwd())

import torch
# Show all values when printing torch tensors.
torch.set_printoptions(threshold=torch.inf) # shows all the values when printing tensors

# Reloads external functions when its code changes into this notebook.
%load_ext autoreload
%autoreload 1
# %aimport will only reload those files
# %aimport utils.bboxes

/Users/tonyavis/Main/AI_public_projects/object_detection/yolo_v1_taco


In [2]:
from yolov1.dataset import Dataset
from utils.load_config import load_config
from utils.setup_transforms import setup_transforms

In [3]:
config = load_config()
transforms = setup_transforms(config.IMAGE_SIZE)

In [21]:
d = Dataset(
    S=config.S,
    B=config.B,
    C=config.C,
    whichDataset=config.WHICH_DATASET,
    transforms=transforms,
)
img, label = d.__getitem__(0)
img.shape, label.shape 

Checking csv dataframe file: csv dataframe file already exists.


(torch.Size([3, 448, 448]), torch.Size([7, 7, 28]))

> Make the label shape look like output we will pass the label as if its the models output and the true label to test IOU

In [22]:
# both are the labeled data
pred = label.unsqueeze(dim=0)
label = label.unsqueeze(dim=0)
C, S, B = config.C, config.S, config.B

In [ ]:
b_s = pred.size(0)


# ==> 1: Slice out the coordinates we need.
#       Layout: C=18, B=2.
#            [c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, pc1, x1,y1,w1,h1, pc2, x2,y2,w2,h2]
#   Get class predictions index 0-17. (n_s, S, S, C)
pred_cls = pred[..., :C]

# pred_cls = torch.softmax(pred_cls, dim=-1)


label_cls = label[..., :C]

#   Get bbox1 pc, and x,y,w,h
pred_box1_pc = pred[
    ..., C : C + 1
]  # Get bbox1 probability score index(18) pc_1. Shape: (64, 7, 7, 1)
pred_box1_xywh = pred[
    ..., C + 1 : C + 5
]  # Get bbox1 x,y,w,h coordinates indexes(19, 20, 21, 22). Shape: (64, 7, 7, 4)

#   Get bbox2 pc, and x,y,w,h
pred_box2_pc = pred[
    ..., C + 5 : C + 6
]  # Get bbox1 probability score @ index(23) pc_2. Shape: (64, 7, 7, 1)
pred_box2_xywh = pred[
    ..., C + 6 : C + 10
]  # Get bbox2 x,y,w,h coordinates @ indexes(24, 25, 26, 27). Shape: (64, 7, 7, 4)

# Get label data, only bbox1 is filled for labeled data, pc (index) which the object=1 and so is the class prediction index (0-17).
label_pc = label[..., C : C + 1]  # Shape: (64, 7, 7, 1)
label_xywh = label[..., C + 1 : C + 5]  # Shape: (64, 7, 7, 4)

In [24]:
from utils.intersection_over_union import intersection_over_union

In [25]:
iou_b1 = intersection_over_union(pred_box1_xywh, label_xywh)  # (64, 7, 7, 1)
# iou_b2 = intersection_over_union(pred_box2_xywh, label_xywh)  # (64, 7, 7, 1)

In [26]:
iou_b1

tensor([[[[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.9990],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [1.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]]]])